# Flask Playground

### This notebook containers Selenium Simulation Code that would preform the data capture from ecWIN.

In [1]:
import sys
import json

from pymongo import MongoClient
from bson.objectid import ObjectId
from influxdb import InfluxDBClient
from influxdb import DataFrameClient


import pandas as pd
import numpy as np
# import matplotlib.pyplot as plt

import time
import datetime

from fuzzywuzzy import fuzz
from fuzzywuzzy import process
from difflib import get_close_matches 

## Configure Mongodb

In [17]:
mongoClient = MongoClient('localhost',27017)
db = mongoClient.witsECOMP

## Configure InfluxDB

In [2]:
databaseName = "timeseriesdata"
influxClient = DataFrameClient("localhost", 8086, "root", "root", database=databaseName)
influxClient.create_database(databaseName)

## Read in time Series from excel spreadsheet
Note: this process can take a long time (2 to 3 minutes!).

In [3]:
df = pd.DataFrame()
spreadsheet = pd.read_excel('/home/chris/Desktop/Wits Data/WITS ecWIN7 Data/WITS ecWIN7 Data 2018.xlsx',
                            sheet_name='WITS ecWIN7 Data 2018',
                            index_col="DateTime"
                           )
# print(spreasheet)

## Read in building names and geojson information
These files were manually created by generating geojson from http://geojson.io/ and identifying the corisponding building names.

In [14]:
buildingSensorNames = spreadsheet.columns
print(buildingSensorNames)

Index(['WITS 13 Jubilee Road_kVA', 'WITS 13 Jubilee Road_kVarh',
       'WITS 13 Jubilee Road_kWh', 'WITS 3 Jubilee Road_kVA',
       'WITS 3 Jubilee Road_kVarh', 'WITS 3 Jubilee Road_kWh',
       'WITS 5 Jubilee Road_kVA', 'WITS 5 Jubilee Road_kVarh',
       'WITS 5 Jubilee Road_kWh', 'WITS 9 Jubilee Road_kVA',
       ...
       'WITS WC Village Zesti Lemonz_kWh', 'WITS WC WITS Club GEN 100 kVa_kVA',
       'WITS WC WITS CLUB GENERATOR 100 kVa_kVarh',
       'WITS WC WITS CLUB GENERATOR 100 kVa_kWh', 'WITS WC Wits Club_kVA',
       'WITS WC Wits Club_kVarh', 'WITS WC Wits Club_kWh',
       'WITS Wits Club Summation_kVA', 'WITS Wits Club Summation_kVarh',
       'WITS Wits Club Summation_kWh'],
      dtype='object', length=986)


In [74]:
loadedGeoJson = {}
with open('../../Assets/geojson/witsMainCampusGeojson.json') as json_data:
    loadedGeoJson = json.load(json_data)
    print(loadedGeoJson)

{'type': 'FeatureCollection', 'features': [{'type': 'Feature', 'properties': {'stroke': '#555555', 'stroke-width': 2, 'stroke-opacity': 1, 'fill': '#555555', 'fill-opacity': 0.5, 'buildingName': 'Chamber of Mines'}, 'geometry': {'type': 'Polygon', 'coordinates': [[[28.026670217514038, -26.19134352734578], [28.026729226112362, -26.191767125729683], [28.026643395423886, -26.191786380165073], [28.026697039604187, -26.19204150113358], [28.0270779132843, -26.19199336514455], [28.027067184448242, -26.19195485633902], [28.02739977836609, -26.19192116112372], [28.027292490005493, -26.191271322922137], [28.026670217514038, -26.19134352734578]]]}}, {'type': 'Feature', 'properties': {'stroke': '#555555', 'stroke-width': 2, 'stroke-opacity': 1, 'fill': '#555555', 'fill-opacity': 0.5, 'buildingName': 'Flower Hall'}, 'geometry': {'type': 'Polygon', 'coordinates': [[[28.025929927825928, -26.19151200416024], [28.025962114334106, -26.191950042737417], [28.026278614997864, -26.191901906710605], [28.0261

In [79]:
for feature in loadedGeoJson["features"]:
    buildingName = feature["properties"]["buildingName"]
    mostLiklySensor = get_close_matches(buildingName + "_kVA", buildingSensorNames)[0]
    print(buildingName + "->" +mostLiklySensor)
    for reading in 

# list(mostLiklyBuildings)

Chamber of Mines->WITS WC Chamber of Mines Total_kVA
Flower Hall->WITS WC Flower Hall_kVA
Ithemba Labs High Voltage->WITS Ithemba Labs High Voltage_kVA
Wits Science Stadium->WITS WC Science Staduim TRF 2_kVA
Mathematical Science Building->WITS WC Maths Science Building_kVA
Zesti Lemonz->WITS EC PDH Zesti Lemon_kVA
FNB Building->WITS WC FNB Building Total_kVA
PEC PIMD Office Block->WITS PEC PIMD Office Block_kVA
DJ Du Plesis->WITS WC DJ du Plessis Building_kVA
West Campus Village->WITS Campus Lodge_kVA
Convocation Dining Hall->WITS WC Convocation Dining Hall GEN 125 kVA_kVA
Barnato Residence->WITS WC Barnato Sub Residence A - D_kVA
David Webster Hall->WITS WC David Webster Hall_kVA
The Barns->WITS WC The Barns_kVA
Olives and Plates->WITS WBS Olives & Plates Meter 2_kVA
Johannesburg Planetarium->WITS EC Planetarium_kVA
Wartenweilier Library->WITS EC Wartenweiler Library_kVA
Origins Centre->WITS EC Origin Centre_kVA
Chemistry Building->WITS EC Physics Building_kVA
Physics Building->WITS E

[('WITS EC Central Block 1_kVA', 90),
 ('WITS EC Central Block 1_kVarh', 90),
 ('WITS EC Central Block 1_kWh', 90),
 ('WITS EC Central Block 2_kVA', 90),
 ('WITS EC Central Block 2_kVarh', 90),
 ('WITS EC Central Block 2_kWh', 90),
 ('WITS Knockando Block H_kVA', 90),
 ('WITS Knockando Block H_kVarh', 90),
 ('WITS Knockando Block H_kWh', 90),
 ('WITS PEC Conacher Block TRF 1_kVA', 90),
 ('WITS PEC Conacher Block TRF 1_kVarh', 90),
 ('WITS PEC Conacher Block TRF 1_kWh', 90),
 ('WITS PEC Conacher Block TRF 2_kVA', 90),
 ('WITS PEC Conacher Block TRF 2_kVarh', 90),
 ('WITS PEC Conacher Block TRF 2_kWh', 90),
 ('WITS PEC Marang Block Trf 2_kVA', 90),
 ('WITS PEC Marang Block Trf 2_kVarh', 90),
 ('WITS PEC Marang Block Trf 2_kWh', 90),
 ('WITS PEC Marang Block_kVA', 90),
 ('WITS PEC Marang Block_kVarh', 90)]

In [10]:
loadedCampusInfo = {}
with open('../../Assets/otherinformation/campusInfomation.json') as json_data:
    loadedCampusInfo = json.load(json_data)
    print(loadedCampusInfo)

{'name': 'Main campus', 'center': ['28.0240, -26.18701'], 'numberOfStudents': 25000, 'averageWeekChart': {}, 'piDistribution': {}, 'sunburstDistribution': {}, 'streamGraph': {}, 'totalSensors': 2000}


In [11]:
campusInfoStore = db.campusInfo
campusInfo_id = campusInfoStore.insert_one(loadedCampusInfo).inserted_id

NameError: name 'db' is not defined

In [7]:
list(campusInfoStore.find({}))

[{'_id': ObjectId('5bbb60adb2831017c6117dbe'),
  'name': 'Main campus',
  'center': ['28.0240, -26.18701'],
  'numberOfStudents': 25000,
  'averageWeekChart': {},
  'piDistribution': {},
  'sunburstDistribution': {},
  'streamGraph': {},
  'totalSensors': 2000}]

In [8]:
loadedGeoJson = {}
with open('../../Assets/geojson/witsMainCampusGeojson.json') as json_data:
    loadedGeoJson = json.load(json_data)
    print(loadedGeoJson)

{'features': [{'geometry': {'coordinates': [[[28.026670217514038, -26.19134352734578], [28.026729226112362, -26.191767125729683], [28.026643395423886, -26.191786380165073], [28.026697039604187, -26.19204150113358], [28.0270779132843, -26.19199336514455], [28.027067184448242, -26.19195485633902], [28.02739977836609, -26.19192116112372], [28.027292490005493, -26.191271322922137], [28.026670217514038, -26.19134352734578]]], 'type': 'Polygon'}, 'properties': {'id': '0', 'buildingName': 'NOT Chamber Of Mines', 'style': {'color': '#999', 'fillColor': '#E04E5C', 'fillOpacity': 0.7, 'opacity': 1, 'weight': 2}}, 'type': 'Feature'}, {'geometry': {'coordinates': [[[28.025929927825928, -26.19151200416024], [28.025962114334106, -26.191950042737417], [28.026278614997864, -26.191901906710605], [28.026192784309387, -26.19148312243793], [28.025929927825928, -26.19151200416024]]], 'type': 'Polygon'}, 'properties': {'id': '1', 'buildingName': 'Chamber Of Mines', 'style': {'color': '#999', 'fillColor': '#

In [9]:
geojsonStore = db.geojson
geojson_id = geojsonStore.insert_one(loadedGeoJson).inserted_id

In [10]:
list(geojsonStore.find({}))

[{'_id': ObjectId('5bbb60adb2831017c6117dbf'),
  'features': [{'geometry': {'coordinates': [[[28.026670217514038,
        -26.19134352734578],
       [28.026729226112362, -26.191767125729683],
       [28.026643395423886, -26.191786380165073],
       [28.026697039604187, -26.19204150113358],
       [28.0270779132843, -26.19199336514455],
       [28.027067184448242, -26.19195485633902],
       [28.02739977836609, -26.19192116112372],
       [28.027292490005493, -26.191271322922137],
       [28.026670217514038, -26.19134352734578]]],
     'type': 'Polygon'},
    'properties': {'id': '0',
     'buildingName': 'NOT Chamber Of Mines',
     'style': {'color': '#999',
      'fillColor': '#E04E5C',
      'fillOpacity': 0.7,
      'opacity': 1,
      'weight': 2}},
    'type': 'Feature'},
   {'geometry': {'coordinates': [[[28.025929927825928, -26.19151200416024],
       [28.025962114334106, -26.191950042737417],
       [28.026278614997864, -26.191901906710605],
       [28.026192784309387, -26.19

In [11]:
BuildingChartInformation = {}
with open('../../Assets/ChartInformation/BuildingChartInformation.json') as json_data:
    BuildingChartInformation = json.load(json_data)
    print(BuildingChartInformation)

{'buildingId': '0', 'dayInformation': {'LastDay': [20, 14, 25, 16, 18, 22, 19, 20, 14, 25, 16, 18, 22, 19, 20, 14, 25, 16, 18, 22, 19, 21, 23, 24], 'AverageDay': [20, 14, 25, 16, 18, 22, 19, 20, 14, 25, 16, 18, 22, 19, 20, 14, 25, 16, 18, 22, 19, 21, 23, 24], 'CampusAverageDay': [20, 14, 25, 16, 18, 22, 19, 20, 14, 25, 16, 18, 22, 19, 20, 14, 25, 16, 18, 22, 19, 21, 23, 24], 'CampusAverageDayNormalized': [20, 14, 25, 16, 18, 22, 19, 20, 14, 25, 16, 18, 22, 19, 20, 14, 25, 16, 18, 22, 19, 21, 23, 24]}, 'weekInformation': {'LastWeek': [20, 14, 25, 16, 18, 22, 19], 'AverageWeek': [19, 14, 22, 14, 16, 19, 15], 'CampusAverageWeek': [28, 22, 30, 28, 28, 31, 22], 'CampusAverageWeekNormalized': [25, 19, 25, 26, 26, 28, 18]}, 'yearInformation': {'LastYear': [20, 14, 25, 16, 18, 22, 19], 'AverageYear': [19, 14, 22, 14, 16, 19, 15], 'CampusAverageYear': [28, 22, 30, 28, 28, 31, 22], 'CampusAverageYearNormalized': [25, 19, 25, 26, 26, 28, 18]}, 'histogramInformation': {}}


In [12]:
buildingInformationStore = db.buildingInformation
buildingInformation_id = buildingInformationStore.insert_one(BuildingChartInformation).inserted_id

In [13]:
list(buildingInformationStore.find({"buildingId": "0"}))

[{'_id': ObjectId('5bbb60adb2831017c6117dc0'),
  'buildingId': '0',
  'dayInformation': {'LastDay': [20,
    14,
    25,
    16,
    18,
    22,
    19,
    20,
    14,
    25,
    16,
    18,
    22,
    19,
    20,
    14,
    25,
    16,
    18,
    22,
    19,
    21,
    23,
    24],
   'AverageDay': [20,
    14,
    25,
    16,
    18,
    22,
    19,
    20,
    14,
    25,
    16,
    18,
    22,
    19,
    20,
    14,
    25,
    16,
    18,
    22,
    19,
    21,
    23,
    24],
   'CampusAverageDay': [20,
    14,
    25,
    16,
    18,
    22,
    19,
    20,
    14,
    25,
    16,
    18,
    22,
    19,
    20,
    14,
    25,
    16,
    18,
    22,
    19,
    21,
    23,
    24],
   'CampusAverageDayNormalized': [20,
    14,
    25,
    16,
    18,
    22,
    19,
    20,
    14,
    25,
    16,
    18,
    22,
    19,
    20,
    14,
    25,
    16,
    18,
    22,
    19,
    21,
    23,
    24]},
  'weekInformation': {'LastWeek': [20, 14, 25, 16, 18, 22, 19],
  